In [1]:
import requests
from scrapy import Selector
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm
import undetected_chromedriver as uc

In [2]:
driver = webdriver.Chrome()
driver.get('https://www.amazon.sa/-/en/b/ref=mh_s9_acss_cg_aeappsbc_3e1_w?node=12463333031&pf_rd_m=A2XPWB6MYN7ZDK&pf_rd_s=mobile-hybrid-3&pf_rd_r=SNF3DAYM6MDVDD51PE64&pf_rd_t=30901&pf_rd_p=65de6af7-be4c-43ed-9007-4406865851b0&pf_rd_i=26449543031')

Error getting version of chromedriver 115. Retrying with chromedriver 114 (attempt 1/5)


In [6]:
Products = {'URL':[],'Name':[],'Price':[],'Brand':[],'Ranks':[],'Rating':[],'Reviews':[],'Description':[],'ASIN':[],'Manufecturer':[],'Image_url':[]}

In [7]:
li = []
sel = Selector(text=driver.page_source)
li = sel.xpath('//li[@class="octopus-pc-category-card-v2-item"]//div[@class="a-section octopus-pc-category-card-v2-item-block"]/a/@href').extract() # category link
for j in li:
    driver.get('https://www.amazon.sa/'+j)
    try:
        driver.find_element('xpath','//span[contains(text(),"See all results")]').click()
    except:
        pass
    sel = Selector(text=driver.page_source)
    pages = sel.xpath('//span[@class="s-pagination-item s-pagination-disabled"]/text()').extract_first()
    for i in tqdm(range(1,int(pages)+1)):
        sel = Selector(text=driver.page_source)
        links = ['https://www.amazon.sa/'+i.xpath('.//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]/@href').extract_first() for i in sel.xpath('//div[@data-component-type="s-search-result"]') if i.xpath('.//span[@class="a-color-base"]/text()').extract_first()!='Sponsored']
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[1])
        for l in links:
            driver.get(l)
            time.sleep(1)
            sel2 = Selector(text=driver.page_source)
            Products['URL'].append(l) # Product Url
            Products['Name'].append(sel2.xpath('//span[@id="productTitle"]/text()').extract_first()) # Product Name
            Products['Price'].append(sel2.xpath('//span[@class="a-price-whole"]/text()').extract_first()) # Product Price
            Products['Brand'].append(sel2.xpath('//div[@class="a-section a-spacing-none"]/a/text()').extract()) # Brand
            ranks = sel2.xpath('//span[text()=" Best Sellers Rank: "]/../../../li/span[@class="a-list-item"]//text()').extract()
            ranks = ' '.join([item.strip() for item in ranks if item.strip()])
            if ranks==None:
                ra = sel2.xpath('//th[contains(text()," Best Sellers Rank ")]/following-sibling::td//span/text()').extract()
                ra = ' '.join([item.strip() for item in ra if item.strip()])
                nk = sel2.xpath('//th[contains(text()," Best Sellers Rank ")]/following-sibling::td//a/text()').extract()[1:][0]
                rank = ra + nk
                Products['Ranks'].append(rank)
            else:
                Products['Ranks'].append(ranks)
            Products['Rating'].append(sel2.xpath('//span[@class="a-icon-alt"]/text()').extract_first()) # Rating
            if sel2.xpath('//span[@id="acrCustomerReviewText"]/text()').extract_first()!=None:
                Products['Reviews'].append(sel2.xpath('//span[@id="acrCustomerReviewText"]/text()').extract_first().split()[0])# Number of Reviews
            else:
                Products['Reviews'].append(None)
            Products['Description'].append(''.join(sel2.xpath('.//ul[@class="a-unordered-list a-vertical a-spacing-mini"]//text()').extract())) # Description
            if sel2.xpath('//th[contains(text()," ASIN ")]/following-sibling::td/text()').extract_first()==None:
                Products['ASIN'].append(sel2.xpath('//th[contains(text()," ASIN ")]/following-sibling::td/text()').extract_first())
            else:
                Products['ASIN'].append(' '.join(sel2.xpath('//th[contains(text()," ASIN ")]/following-sibling::td/text()').extract_first().split()).replace('\u200e',''))
            if sel2.xpath('//th[contains(text()," Manufacturer ")]/following-sibling::td/text()').extract_first()==None:
                Products['Manufecturer'].append(None)
            else:
                Products['Manufecturer'].append(' '.join(sel2.xpath('//th[contains(text()," Manufacturer ")]/following-sibling::td/text()').extract_first().split()).replace('\u200e',''))
            Products['Image_url'].append(sel2.xpath('//div[@id="imgTagWrapperId"]/img/@src').extract_first()) # Image url
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        driver.find_element('xpath','//a[contains(text(),"Next")]').click()
        time.sleep(2)

  0%|                                                                                          | 0/400 [00:29<?, ?it/s]


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.110)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00D88893+48451]
	(No symbol) [0x00D1B8A1]
	(No symbol) [0x00C25058]
	(No symbol) [0x00C0D073]
	(No symbol) [0x00C6DEBB]
	(No symbol) [0x00C7BFD3]
	(No symbol) [0x00C6A0B6]
	(No symbol) [0x00C47E08]
	(No symbol) [0x00C48F2D]
	GetHandleVerifier [0x00FE8E3A+2540266]
	GetHandleVerifier [0x01028959+2801161]
	GetHandleVerifier [0x0102295C+2776588]
	GetHandleVerifier [0x00E12280+612144]
	(No symbol) [0x00D24F6C]
	(No symbol) [0x00D211D8]
	(No symbol) [0x00D212BB]
	(No symbol) [0x00D14857]
	BaseThreadInitThunk [0x757D00C9+25]
	RtlGetAppContainerNamedObjectPath [0x77977B4E+286]
	RtlGetAppContainerNamedObjectPath [0x77977B1E+238]


In [8]:
df = pd.DataFrame(Products)

In [9]:
df.to_excel('Appliances.xlsx',index=False)